In [1]:
from pyspark import SparkContext, SparkConf
from operator import add

conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


from pyspark.sql.types import *

In [3]:
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')
# df = sqlContext.read.format('com.databricks.spark.csv').load('./moviedata/rating.csv')

In [39]:
all_movieids = df_movies.map(lambda x: x.split(",")).map(lambda x: (x[0], 0))
#    map(lambda y: (int(y[1]),float(y[2])))
# .\
#     map(lambda x: (x[0], x[1])

In [45]:
movieid_rating = df_rating.map(lambda x: x.split(",")).map(lambda x:(x[1], 1)).reduceByKey(lambda x,y: 1)
#     reduce(lambda x,y:(x+y))

In [46]:
movieid_rating.count()

1587

In [47]:
f = all_movieids + movieid_rating

In [48]:
f.take(10)

[(u'279', 0),
 (u'280', 0),
 (u'281', 0),
 (u'282', 0),
 (u'283', 0),
 (u'284', 0),
 (u'285', 0),
 (u'286', 0),
 (u'288', 0),
 (u'289', 0)]

In [58]:
def select_value_zero(p):
    return p[1] == 0

In [59]:
g = f.reduceByKey(lambda x, y: x+y).filter(select_value_zero)

In [60]:
g.take(10)

[(u'1667', 0),
 (u'1663', 0),
 (u'1670', 0),
 (u'1666', 0),
 (u'1309', 0),
 (u'1669', 0),
 (u'1668', 0),
 (u'1310', 0),
 (u'1682', 0)]

In [61]:
movieid_not_rated = g.map(lambda x: x[0])

In [62]:
movieid_not_rated.collect()

[u'1667',
 u'1663',
 u'1670',
 u'1666',
 u'1309',
 u'1669',
 u'1668',
 u'1310',
 u'1682']